In [1]:
#@title Imports

import numpy as np

import tensorflow as tf
from tensorflow import keras

import tensorflow_datasets as tfds

import sklearn as sk
from sklearn.feature_extraction.text import CountVectorizer

import os
import nltk

import matplotlib.pyplot as plt

import re
import textwrap

from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration
from transformers import GPT2Tokenizer, TFOPTForCausalLM

2023-10-06 10:03:00.890898: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/peeti_mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Building a Seq2Seq model for Translation using RNNs with and without Attention

### Downloading and pre-processing Data


Let's get the data. Just like the Keras tutorial, we will use http://www.manythings.org as the source for the parallel corpus, but we will use German.  Machine translation requires sentence pairs for training, that is individual sentences in German and the corresponding sentence in English.

In [3]:
!!curl -O http://www.manythings.org/anki/deu-eng.zip
!!unzip deu-eng.zip

['Archive:  deu-eng.zip',
 'replace deu.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL',
 '(EOF or read error, treating as "[N]one" ...)']

Note these numbers are much smaller than the real world plus I am working on cpu machine

In [4]:
embed_dim = 100  # Embedding dimensions for vectors and LSTMs.
num_samples = 10000  # Number of examples to consider.

# Path to the data txt file on disk.
data_path = "deu.txt"

# Vocabulary sizes that we'll use:
english_vocab_size = 2000
german_vocab_size = 3000

Next we need to format the input by using nltk for the tokenization.

using CountVectorizer to create a vocabulary from the most frequent words in each language

In [5]:
input_texts = []
target_texts = []

max_input_length = -1
max_output_length = -1


with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")

    tokenized_source_text = nltk.word_tokenize(input_text, language='english')
    tokenized_target_text = nltk.word_tokenize(target_text, language='german')

    if len(tokenized_source_text) > max_input_length:
      max_input_length = len(tokenized_source_text)

    if len(tokenized_target_text) > max_output_length:
      max_output_length = len(tokenized_target_text)


    source_text = (' '.join(tokenized_source_text)).lower()
    target_text = (' '.join(tokenized_target_text)).lower()

    input_texts.append(source_text)
    target_texts.append(target_text)

vectorizer_english = CountVectorizer(max_features=english_vocab_size)
vectorizer_english.fit(input_texts)
vocab_english = vectorizer_english.get_feature_names_out()

vectorizer_german = CountVectorizer(max_features=german_vocab_size)
vectorizer_german.fit(target_texts)
vocab_german = vectorizer_german.get_feature_names_out()

print('Maximum source input length: ', max_input_length)
print('Maximum target output length: ', max_output_length)

Maximum source input length:  6
Maximum target output length:  10


In [6]:
# lets look at a few input words

input_texts[:2]

['go .', 'hi .']

In [7]:
# here with german translation 

target_texts[:2]

['geh .', 'hallo !']

from our source and target sequences above, we set our max lengths 6 and 11, respectively. As we will add start and end tokens (\<s> and \</s>) to our decoder side we will set the respective max lengths to: 

In [8]:
max_encoder_seq_length = 6
max_decoder_seq_length = 13 #11 + start + end

Next, we create the dictionaries translating between integer ids and tokens for both source (English) and target (German).

In [9]:
source_id_vocab_dict = {}
source_vocab_id_dict = {}

for sid, svocab in enumerate(vocab_english):
  source_id_vocab_dict[sid] = svocab
  source_vocab_id_dict[svocab] = sid

source_id_vocab_dict[english_vocab_size] = "<unk>"
source_id_vocab_dict[english_vocab_size + 1] = "<pad>"

source_vocab_id_dict["<unk>"] = english_vocab_size
source_vocab_id_dict["<pad>"] = english_vocab_size + 1

target_id_vocab_dict = {}
target_vocab_id_dict = {}

for tid, tvocab in enumerate(vocab_german):
  target_id_vocab_dict[tid] = tvocab
  target_vocab_id_dict[tvocab] = tid

# Add unknown token plus start and end tokens to target language

target_id_vocab_dict[german_vocab_size] = "<unk>"
target_id_vocab_dict[german_vocab_size + 1] = "<start>"
target_id_vocab_dict[german_vocab_size + 2] = "<end>"
target_id_vocab_dict[german_vocab_size + 3] = "<pad>"

target_vocab_id_dict["<unk>"] = german_vocab_size
target_vocab_id_dict["<start>"] = german_vocab_size + 1
target_vocab_id_dict["<end>"] = german_vocab_size + 2
target_vocab_id_dict["<pad>"] = german_vocab_size + 3

Lastly, we need to create the training and test data that will feed into our two models. It is convenient to define a small function for that that also takes care off padding and adding start/end tokens on the decoder side.

Notice that we need to create three sequences of vocab ids: inputs to the encoder (starting language), inputs to the decoder (output language, for the preceding tokens in the output sequence) and labels for the decoder (the correct next word to predict at each time step in the output, which is shifted one over from the inputs to the decoder).

In [10]:
def convert_text_to_data(texts, 
                         vocab_id_dict, 
                         max_length=20, 
                         type=None,
                         train_test_vector=None,
                         samples=100000):
  
  if type == None:
    raise ValueError('\'type\' is not defined. Please choose from: input_source, input_target, output_target.')
  
  train_data = []
  test_data = []

  for text_num, text in enumerate(texts[:samples]):

    sentence_ids = []

    for token in text.split():

      if token in vocab_id_dict.keys():
        sentence_ids.append(vocab_id_dict[token])
      else:
        sentence_ids.append(vocab_id_dict["<unk>"])
    
    vocab_size = len(vocab_id_dict.keys())
    
    # Depending on encoder/decoder and input/output, add start/end tokens.
    # Then add padding.
    
    if type == 'input_source':
      ids = (sentence_ids + [vocab_size - 1] * max_length)[:max_length]

    elif type == 'input_target':
      ids = ([vocab_size -3] + sentence_ids + [vocab_size - 2] + [vocab_size - 1] * max_length)[:max_length]

    elif type == 'output_target':
      ids = (sentence_ids + [vocab_size - 2] + [vocab_size -1] * max_length)[:max_length]

    if train_test_vector is not None and not train_test_vector[text_num]:
      test_data.append(ids)
    else:
      train_data.append(ids)


  return np.array(train_data), np.array(test_data)


train_test_split_vector = (np.random.uniform(size=10000) > 0.2)

train_source_input_data, test_source_input_data = convert_text_to_data(input_texts, 
                                                                       source_vocab_id_dict,
                                                                       type='input_source',
                                                                       max_length=max_encoder_seq_length,
                                                                       train_test_vector=train_test_split_vector)

train_target_input_data, test_target_input_data = convert_text_to_data(target_texts,
                                                                       target_vocab_id_dict,
                                                                       type='input_target',
                                                                       max_length=max_decoder_seq_length,
                                                                       train_test_vector=train_test_split_vector)

train_target_output_data, test_target_output_data = convert_text_to_data(target_texts,
                                                                         target_vocab_id_dict,
                                                                         type='output_target',
                                                                         max_length=max_decoder_seq_length,
                                                                         train_test_vector=train_test_split_vector)




**T5 system** was introduced for generating text with transformer. This model uses both the encoder and the decoder configurations of transformers and connects them together. A big difference with this model is that it designed to accept text as an input and produce text as an output for a number of different tasks ranging from summarization and question answering to classification. The system needs to be told which task to perform as the first part of the input text.

In [11]:
t5_model = TFT5ForConditionalGeneration.from_pretrained('t5-large')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-large')

t5_model.summary()

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/Users/peeti_mac/opt/anaconda3/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:220: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  32899072  
                                                                 
 encoder (TFT5MainLayer)     multiple                  334939648 
                                                                 
 decoder (TFT5MainLayer)     multiple                  435627520 
                                                                 
Total params: 737668096 (2.75 GB)
Trainable params: 737668096 (2.75 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


lets start with some examples


In [12]:
ARTICLE = ("Oh boy, what a lengthy and cumbersome excercise this was. " \
           "I had to look into every detail, check everything twice, " \
           " and then compare to prior results. Because of this tediousness " \
           " and extra work my homework was 2 days late.")

we need to specify the task we want T5 to perform and include it at the begining of the input text. We add a task prompt to the begining of our input. Because we are summarizing, we add the word summarize: to the begining of our input.

In [13]:
t5_input_text = "summarize: " + ARTICLE
t5_inputs = t5_tokenizer([t5_input_text], return_tensors='tf')

Not great. But let's get more sophisticated and prescribe a minimum length and use beam search to generate multiple outputs.  We also indicate the maximum length the output should be.  Finally, in order to reduce repetitive output we tell the model to avoid output that repeats trigrams (three word groupings).

In [14]:
t5_summary_ids = t5_model.generate(t5_inputs['input_ids'])

print([t5_tokenizer.decode(g, skip_special_tokens=True,
                           clean_up_tokenization_spaces=False)
       for g in t5_summary_ids])

/Users/peeti_mac/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/tf_utils.py:834: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2023-10-06 10:04:29.634386: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ff046945ce0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-10-06 10:04:29.634838: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-10-06 10:04:29.638412: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 10:04:29.736095: I tensorflow/comp

['homework was a lengthy and cumbersome excercise . because of this tedious']


Not great. But let's get more sophisticated and prescribe a minimum length and use beam search to generate multiple outputs.  We also indicate the maximum length the output should be.  Finally, in order to reduce repetitive output we tell the model to avoid output that repeats trigrams (three word groupings).

In [15]:
t5_input_text = "translate English to German: " + ARTICLE
t5_inputs = t5_tokenizer([t5_input_text], return_tensors='tf')

In [16]:
t5_summary_ids = t5_model.generate(t5_inputs['input_ids'],
                                   num_beams=3,
                                   no_repeat_ngram_size=3,
                                   min_length=10,
                                   max_length=40)
                             
print([t5_tokenizer.decode(g, skip_special_tokens=True, 
                           clean_up_tokenization_spaces=False) for g in t5_summary_ids])

['Ich habe es nicht geschafft, meinen ersten Test zu schreiben, da ich nicht genügend Zeit hatte, um meinen Test zu bearbeiten.']


In [17]:
t5_input_text = "translate English to German: That was really not very good today; it was too difficult to solve."
t5_inputs = t5_tokenizer([t5_input_text], return_tensors='tf')

t5_summary_ids = t5_model.generate(t5_inputs['input_ids'],
                                   num_beams=3,
                                   no_repeat_ngram_size=3,
                                   min_length=10,
                                   max_length=40)
                             
print([t5_tokenizer.decode(g, skip_special_tokens=True, 
                           clean_up_tokenization_spaces=False) for g in t5_summary_ids])

['Das war heute wirklich nicht sehr gut; es war zu schwierig zu lösen.']


Hmm... output language fluency is very good. But take the German output and feed it in to translate.google.com and see what this means. Is it anything like its English input? This hallucination might be mitigated by changing some of the hyperparameters like num_beams.

Is a shorter example more accurate?  Maybe.